In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from tqdm.notebook import tqdm
import copy

In [2]:
use_cuda = torch.cuda.is_available()

def w(v):
    if use_cuda:
        # Returns a copy of the object in cuda memory 
        return v.cuda()

    # Otherwise returns the object itself
    return v

In [3]:
def detach_var(v):
    var = w(v.data)
    var.requires_grad_()
    var.retain_grad()
    return var

In [4]:
def train(opt_net, meta_opt, target_cls, target_to_opt, unroll, optim_iter, n_epochs, out_mul, training=True):
    if training:
        opt_net.train()
    else:
        opt_net.eval()
        unroll = 1
    
    target = target_cls(training=training)
    optimizee = w(target_to_opt())
    n_params = 0

    for p in optimizee.parameters():
        n_params += int(np.prod(p.size()))

    hidden_states = [w(torch.zeros(n_params, opt_net.hidden_sz)) for _ in range(2)]
    cell_states = [w(torch.zeros(n_params, opt_net.hidden_sz)) for _ in range(2)]
    all_losses_ever = []

    if training:
        meta_opt.zero_grad()

    all_losses = None

    for iteration in range(1, optim_iter + 1):
        loss = optimizee(target)
                    
        if all_losses is None:
            all_losses = loss
        else:
            all_losses += loss
        
        all_losses_ever.append(loss.data.cpu().numpy())
        loss.backward(retain_graph=training)

        offset = 0
        result_params = {}
        hidden_states2 = [w(torch.zeros(n_params, opt_net.hidden_sz)) for _ in range(2)]
        cell_states2 = [w(torch.zeros(n_params, opt_net.hidden_sz)) for _ in range(2)]

        for name, p in optimizee.all_named_parameters():
            cur_sz = int(np.prod(p.size()))
            gradients = detach_var(p.grad.view(cur_sz, 1))
            updates, new_hidden, new_cell = opt_net(
                gradients,
                [h[offset:offset+cur_sz] for h in hidden_states],
                [c[offset:offset+cur_sz] for c in cell_states]
            )

            for i in range(len(new_hidden)):
                hidden_states2[i][offset:offset+cur_sz] = new_hidden[i]
                cell_states2[i][offset:offset+cur_sz] = new_cell[i]
                
            result_params[name] = p + updates.view(*p.size()) * out_mul
            result_params[name].retain_grad()
            
            offset += cur_sz
            
        if iteration % unroll == 0:
            if training:
                meta_opt.zero_grad()
                all_losses.backward()
                meta_opt.step()
                
            all_losses = None
                        
            optimizee = w(target_to_opt(**{k: detach_var(v) for k, v in result_params.items()}))
            hidden_states = [detach_var(v) for v in hidden_states2]
            cell_states = [detach_var(v) for v in cell_states2]
            
        else:
            optimizee = w(target_to_opt(**result_params))
            assert len(list(optimizee.all_named_parameters()))
            hidden_states = hidden_states2
            cell_states = cell_states2
            
    return all_losses_ever

In [5]:
def train_optimizer(target_cls, target_to_opt, preproc=False, unroll=20, optim_iter=100, n_epochs=20, n_tests=100, lr=0.001, out_mul=1.0):
    opt_net = w(Optimizer(preproc=preproc))
    meta_opt = optim.Adam(opt_net.parameters(), lr=lr)
    
    best_net = None
    best_loss = 1e10
    
    for _ in tqdm(range(n_epochs), 'epochs'):
        for _ in tqdm(range(20), 'iterations'):
            train(opt_net, meta_opt, target_cls, target_to_opt, unroll, optim_iter, n_epochs, out_mul, training=True)
        
        loss = (np.mean([
            np.sum(train(opt_net, meta_opt, target_cls, target_to_opt, unroll, optim_iter, n_epochs, out_mul, training=False))
            for _ in tqdm(range(n_tests), 'tests')
        ]))

        print(loss)

        if loss < best_loss:
            print(best_loss, loss)
            best_loss = loss
            best_net = copy.deepcopy(opt_net.state_dict())
            
    return best_loss, best_net

In [6]:
class Loss:
    def __init__(self, **kwargs):
        self.W = w(torch.randn(10, 10))
        self.y = w(torch.randn(10))
        
    def get_loss(self, theta):
        return torch.sum((self.W.matmul(theta) - self.y)**2)

class ObjectiveFn(nn.Module):
    def __init__(self, theta=None):
        super().__init__()
        if theta is None:
            self.theta = nn.Parameter(torch.zeros(10))
        else:
            self.theta = theta
        
    def forward(self, target):
        return target.get_loss(self.theta)
    
    def all_named_parameters(self):
        return [('theta', self.theta)]

In [7]:
class Optimizer(nn.Module):
    def __init__(self, preproc=False, hidden_sz=20, preproc_factor=10.0):
        super().__init__()
        self.hidden_sz = hidden_sz
        if preproc:
            self.recurs = nn.LSTMCell(2, hidden_sz)
        else:
            self.recurs = nn.LSTMCell(1, hidden_sz)
        self.recurs2 = nn.LSTMCell(hidden_sz, hidden_sz)
        self.output = nn.Linear(hidden_sz, 1)
        self.preproc = preproc
        self.preproc_factor = preproc_factor
        self.preproc_threshold = np.exp(-preproc_factor)
        
    def forward(self, inp, hidden, cell):
        if self.preproc:
            inp = inp.data
            inp2 = w(torch.zeros(inp.size()[0], 2))
            keep_grads = (torch.abs(inp) >= self.preproc_threshold).squeeze()
            inp2[:, 0][keep_grads] = (torch.log(torch.abs(inp[keep_grads]) + 1e-8) / self.preproc_factor).squeeze()
            inp2[:, 1][keep_grads] = torch.sign(inp[keep_grads]).squeeze()
            inp2[:, 0][~keep_grads] = -1
            inp2[:, 1][~keep_grads] = (float(np.exp(self.preproc_factor)) * inp[~keep_grads]).squeeze()
            inp = w(Variable(inp2))
            
        hidden0, cell0 = self.recurs(inp, (hidden[0], cell[0]))
        hidden1, cell1 = self.recurs2(hidden0, (hidden[1], cell[1]))
        return self.output(hidden1), (hidden0, hidden1), (cell0, cell1)

In [ ]:
loss, quad_optimizer = train_optimizer(Loss, ObjectiveFn, lr=0.003, n_epochs=10)
print(loss)